In [16]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import math

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Load the data into a Pandas DataFrame
bt = pd.read_csv("Bitcoin Historical Data (2014-2024).csv")

# Convert the date column to datetime format if it's not already in datetime
bt['Date'] = pd.to_datetime(bt['Date'], format='%m/%d/%Y')

# Sort the DataFrame by the 'Date' column in ascending order
bt = bt.sort_values(by='Date', ascending=True)

# Columns information
bt.info()

# Convert 'Price' column to float
bt["Price"] = bt["Price"].apply(lambda x: float(x.replace(',', '')))

# Only use the 'Price' column
bitcoin_prices = bt["Price"]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
bitcoin_prices_scaled = scaler.fit_transform(np.array(bitcoin_prices).reshape(-1, 1))

# Splitting the data set into train and test split
X, y = [], []
for i in range(len(bitcoin_prices_scaled) - 100 - 10):
    X.append(bitcoin_prices_scaled[i:i + 100])
    y.append(bitcoin_prices_scaled[i + 100:i + 100 + 10])

y = np.array(y)[:, :, 0].reshape(-1, 10, 1)
X= X = np.array(X).reshape(-1, 100, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create the LSTM model
# model = Sequential()
# model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
# model.add(LSTM(units=50, return_sequences=True))
# model.add(LSTM(units=50))
# model.add(Dense(units=10))  # 10 output nodes for predicting the next 10 days
# model.compile(optimizer='adam', loss='mean_squared_error')

input_shape = (X_train.shape[1], X_train.shape[2])

# Create the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50, return_sequences=True))  # Add this line
model.add(Dense(units=10))  # Adjusted for 10-day output sequence
model.compile(optimizer='adam', loss='mean_squared_error')

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape)




epochs = 100
batch_size = 32

history = model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=1)

# Print the shape of the output from the model
predictions = model.predict(X_train)
print("Output shape:", predictions.shape)

# Print the shape of the labels
print("Labels shape:", y_train.shape)
# for epoch in range(epochs):
#     print("Epoch:", epoch + 1)
#     history = model.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=1)
    
#     # Print the shape of the output from the model
#     predictions = model.predict(X_train)
#     print("Output shape:", predictions.shape)

# # Train the model
# for epoch in range(epochs):
#     print("Epoch:", epoch + 1)
#     for i in range(0, len(X_train), batch_size):
#         batch_X_train = X_train[i:i+batch_size]
#         batch_y_train = y_train[i:i+batch_size]
#         print("Batch:", i//batch_size + 1)
#         print("Batch X_train shape:", batch_X_train.shape)
#         print("Batch y_train shape:", batch_y_train.shape)
#         model.train_on_batch(batch_X_train, batch_y_train)

#                 # Print the shape of the output from the model
#         predictions = model.predict(batch_X_train)
#         print("Output shape:", predictions.shape)

# # Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=32)

# # Evaluate the model
# test_loss = model.evaluate(X_test, y_test)
# print("Test Loss:", test_loss)

# Do the prediction
predictions = model.predict(X_test)

# Inverse transform the predictions
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))

# Convert prediction values to discrete classes
y_pred_classes = np.where(np.diff(predictions, axis=0) > 0, 1, np.where(np.diff(predictions, axis=0) < 0, 0, 2)).reshape(-1)

# Convert true labels to discrete classes
y_true_classes = np.where(np.diff(scaler.inverse_transform(y_test.reshape(-1, 1)), axis=0) > 0, 1,
                          np.where(np.diff(scaler.inverse_transform(y_test.reshape(-1, 1)), axis=0) < 0, 0,
                                   2)).reshape(-1)

# Calculate accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print("Accuracy:", accuracy)

# Plotting the results
plt.plot(predictions, label='Predicted Prices')
plt.plot(scaler.inverse_transform(y_test[:, 0]), label='Actual Prices')
plt.legend()
plt.show()


<class 'pandas.core.frame.DataFrame'>
Index: 3654 entries, 3653 to 0
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      3654 non-null   datetime64[ns]
 1   Price     3654 non-null   object        
 2   Open      3654 non-null   object        
 3   High      3654 non-null   object        
 4   Low       3654 non-null   object        
 5   Vol.      3654 non-null   object        
 6   Change %  3654 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 228.4+ KB
X_train shape: (2835, 100, 1)
y_train shape: (2835, 10, 1)


c:\Users\lucas\anaconda3\envs\dev\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Dimensions must be equal, but are 10 and 100 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](data_1, sequential_15_1/dense_15_1/Add)' with input shapes: [?,10,1], [?,100,10].